# Introdução ao SQL - Manipulando Esquemas

## Recapitulando

Comandos apresentados na aula passada.

## Objetivos

Ao final desta aula o aluno deverá conhecer:

1. A utilidade das VIEWS.
1. As diferenças entre Materialized Views e Views.
1. Como executar um script SQL utilizando a ferramenta psql.

# Views

E se precisarmos usar o resultado de uma query diversas vezes?

Podemos criar uma `VIEW` e acessar esse resultado como se fosse uma tabela!

O resultado das views não são armazenados no banco. A consulta que define a view é executada toda vez que é referenciada por alguma consulta.

Para armazenar os resultados, utiliza-se as <a href="https://www.postgresql.org/docs/9.3/rules-materializedviews.html"> Materialized Views</a>.

## CREATE VIEW ... AS

Criar a view:

    CREATE VIEW quantidade_de_cursos_por_aluno as (
        SELECT id_aluno, count(*)
        from turma_aluno
        group by id_aluno
        having count(*) > 1
    );

Resultado:

<img src="./images/View.png" width="60%" height="60%"/>

Consultar a view:

    SELECT COUNT(*) FROM quantidade_de_cursos_por_aluno;

## CREATE MATERIALIZED VIEW ... AS

    CREATE MATERIALIZED VIEW total_prod_vendido AS (
        SELECT product_id, SUM(quantity) AS total_vendido
        FROM order_details
        GROUP BY product_id
        ORDER BY total_vendido
    );
    
As visões materializadas encontram-se no nó Materialized Views no PgAdmin.

## REFRESH VIEW

Como as visões materializadas de fato armazenam os resultados no banco, é preciso atualiza-las de tempos em tempos.

Considerando a visão materializada do exemplo, se a tabela order_details (utilizada na criação da view) for atualizada, o resultado da view ficará obsoleto.

    REFRESH MATERIALIZED VIEW total_prod_vendido;

## DROP VIEW ...

Quando a análise ficou obsoleta e não precisamos mais desses dados.

Query:

    DROP VIEW quantidade_de_cursos_por_aluno;

## REPLACE VIEW

Vamos atualizar os dados!

Query:

    CREATE OR REPLACE VIEW count_all_view as (
        SELECT id_aluno, count(*) cursos
        from turma_aluno
        group by id_aluno
        having count(*) > 2
    );

## Utilidade prática das Views: 

1 - Views simples:
    
    Prover acesso a dados específicos sem a necessidade de liberar acesso a todo o banco para terceiros.
    
    Dessa forma, não é necessário que terceiros conheçam detalhes sobre o banco, suas tabelas, etc.
    
2 - Views materializadas:

    Permite que o banco faça um cache dos resultados de uma consulta muito custosa.
    
    Não será necessário o gasto de tempo processando a consulta, pois ela já estará pré-processada.